In [28]:
import open_clip
og_model_name = 'laion/CLIP-ViT-B-32-DataComp.XL-s13B-b90K'
model_name = 'hf-hub:' + og_model_name
og_model, _, preproc = open_clip.create_model_and_transforms(model_name)

In [29]:
from vit_prisma.models.base_vit import HookedViT
hooked_model = HookedViT.from_pretrained('open-clip:' + og_model_name, is_timm=False, is_clip=True, fold_ln=False, center_writing_weights=False) # in future, do all models
hooked_model.to("cuda")
hooked_model.eval()

model_id download_pretrained_from_hf: laion/CLIP-ViT-B-32-DataComp.XL-s13B-b90K
Official model name open-clip:laion/CLIP-ViT-B-32-DataComp.XL-s13B-b90K
Converting OpenCLIP weights
model_id download_pretrained_from_hf: laion/CLIP-ViT-B-32-DataComp.XL-s13B-b90K
visual projection shape torch.Size([768, 512])
Setting center_writing_weights to False for OpenCLIP
Setting fold_ln to False for OpenCLIP
Loaded pretrained model open-clip:laion/CLIP-ViT-B-32-DataComp.XL-s13B-b90K into HookedTransformer


HookedViT(
  (embed): PatchEmbedding(
    (proj): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32))
  )
  (hook_embed): HookPoint()
  (pos_embed): PosEmbedding()
  (hook_pos_embed): HookPoint()
  (hook_full_embed): HookPoint()
  (ln_pre): LayerNorm(
    (hook_scale): HookPoint()
    (hook_normalized): HookPoint()
  )
  (hook_ln_pre): HookPoint()
  (blocks): ModuleList(
    (0-11): 12 x TransformerBlock(
      (ln1): LayerNorm(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln2): LayerNorm(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
      )
      (mlp): MLP(
        (hook_pre): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_attn_in): Ho

In [30]:
final_output_hooked, cache = hooked_model.run_with_cache(image.cuda())

In [41]:
cache["hook_embed"].shape

torch.Size([1, 49, 768])

In [32]:
for key in cache:
    print(key)

hook_embed
hook_pos_embed
hook_full_embed
ln_pre.hook_scale
ln_pre.hook_normalized
hook_ln_pre
blocks.0.hook_resid_pre
blocks.0.ln1.hook_scale
blocks.0.ln1.hook_normalized
blocks.0.attn.hook_q
blocks.0.attn.hook_k
blocks.0.attn.hook_v
blocks.0.attn.hook_attn_scores
blocks.0.attn.hook_pattern
blocks.0.attn.hook_z
blocks.0.hook_attn_out
blocks.0.hook_resid_mid
blocks.0.ln2.hook_scale
blocks.0.ln2.hook_normalized
blocks.0.mlp.hook_pre
blocks.0.mlp.hook_post
blocks.0.hook_mlp_out
blocks.0.hook_resid_post
blocks.1.hook_resid_pre
blocks.1.ln1.hook_scale
blocks.1.ln1.hook_normalized
blocks.1.attn.hook_q
blocks.1.attn.hook_k
blocks.1.attn.hook_v
blocks.1.attn.hook_attn_scores
blocks.1.attn.hook_pattern
blocks.1.attn.hook_z
blocks.1.hook_attn_out
blocks.1.hook_resid_mid
blocks.1.ln2.hook_scale
blocks.1.ln2.hook_normalized
blocks.1.mlp.hook_pre
blocks.1.mlp.hook_post
blocks.1.hook_mlp_out
blocks.1.hook_resid_post
blocks.2.hook_resid_pre
blocks.2.ln1.hook_scale
blocks.2.ln1.hook_normalized
blocks

In [2]:
og_model

CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (patch_dropout): Identity()
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): ModuleList(
        (0-11): 12 x ResidualAttentionBlock(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ls_1): Identity()
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): GELU(approximate='none')
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ls_2): Identity()
        )
      )
    )
    (ln_post): LayerNorm((768,), eps=1e-05, elementwise_affine

In [3]:
import torch
from PIL import Image

tokenizer = open_clip.get_tokenizer('ViT-B-32')

image = preproc(Image.open("/workspace/ViT-Prisma/CLIP.png.1")).unsqueeze(0)
text = tokenizer(["a chart", "a diagram", "a dog", "a cat"])

with torch.no_grad(), torch.cuda.amp.autocast():
    image_features = og_model.encode_image(image)
    text_features = og_model.encode_text(text)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)

print("Label probs:", text_probs)  # prints: [[1., 0., 0.]]

Label probs: tensor([[4.1990e-01, 5.7993e-01, 4.0174e-05, 1.3253e-04]])


In [4]:
image_features

tensor([[ 1.4814e-02,  5.6827e-04, -5.0106e-03,  2.5327e-02,  1.1405e-03,
         -5.0535e-02,  2.6294e-02, -3.0264e-02, -2.0377e-02,  4.4621e-02,
         -4.3194e-02, -9.8643e-02, -2.0076e-02,  2.1366e-03, -2.9839e-02,
          9.7225e-03,  1.5830e-02, -2.2068e-02,  2.1483e-02, -1.9164e-02,
         -7.6312e-04,  1.0234e-02,  1.5637e-02,  2.7757e-02,  6.0332e-03,
          1.5497e-02, -9.6189e-03,  3.9546e-02, -1.5755e-02, -5.9842e-03,
         -1.4172e-02,  1.1185e-02,  2.5468e-02, -9.9765e-02, -1.1933e-02,
         -2.3282e-04,  2.0405e-02,  1.2976e-02, -8.7116e-03,  1.4091e-02,
         -1.3447e-01, -3.8007e-02, -2.7824e-02, -9.2259e-03, -1.0193e-02,
         -2.1133e-03, -2.1099e-02,  1.3982e-02, -5.8950e-02, -4.9354e-02,
         -3.5513e-02,  4.2942e-02,  1.1141e-03, -2.2232e-02, -7.1256e-03,
         -5.5039e-03,  1.1019e-01,  7.0468e-02, -1.3597e-02,  4.4308e-03,
          9.5338e-03,  8.4288e-03, -2.7106e-03, -1.1791e-02,  3.5585e-02,
         -4.9919e-02, -5.5466e-02, -7.

In [5]:
text_features

tensor([[ 0.0135, -0.0410,  0.0104,  ..., -0.0148,  0.0418,  0.0083],
        [ 0.0047, -0.0260, -0.0183,  ..., -0.0194,  0.0283, -0.0253],
        [-0.0332, -0.0373, -0.0535,  ..., -0.0360,  0.0324,  0.0046],
        [-0.0196, -0.0310, -0.0072,  ..., -0.0623, -0.0179,  0.0553]])